In [ ]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
from scipy.interpolate import NearestNDInterpolator
from math import * 
plt.rcParams['figure.figsize'] = [8, 6]

GT = plt.imread("./inputs/P2/wait_for_me_daddy_gt.png")
GT *= 255 
GT = GT.astype(np.uint8)

In [ ]:
def NearestInterpolation(img, scale):
    new_img = np.zeros((int(img.shape[0]*scale), int(img.shape[1]*scale)),dtype=np.float32)
    
    for i in range(new_img.shape[0]):
        for j in range(new_img.shape[1]):
            start_i = int(i/scale)
            start_j = int(j/scale)
            dis = np.inf
            
            for row in range(2):
                for col in range(2):
                    pre_i = abs((start_i + row) - i/scale)
                    pre_j = abs((start_j + col)- j/scale)
                    _d = sqrt(pre_i**2+pre_j**2)
                    if _d < dis:
                        I = start_i + row
                        J = start_j + col
                        dis = _d
            
            new_img[i,j] = img[I,J]

    new_img *= 255 
    new_img = new_img.astype(np.uint8)
    return new_img

img = plt.imread("./inputs/P2/wait_for_me_daddy_lr.png")
new_img1 = NearestInterpolation(img,2)
plt.imsave("./outputs/P02/a-NI-Result.png",new_img1,cmap="gray")

In [ ]:
def bilinear_single_inter(x, y, img):
    if x < 0 or y < 0:
        raise ValueError("x and y pixel position have to be positive!")
    if img.shape[1] - 1 < x or img.shape[0] - 1 < y:
        print(x, img.shape[1] - 1,y,img.shape[0] - 1)
        raise ValueError(
            "x and y pixel position have to be smaller than image" "dimensions."
        )

    x_rounded_up = int(ceil(x))
    x_rounded_down = int(floor(x))
    y_rounded_up = int(ceil(y))
    y_rounded_down = int(floor(y))

    ratio_x = x - x_rounded_down
    ratio_y = y - y_rounded_down

    interpolate_x1 = img[y_rounded_down,x_rounded_down] * (1-ratio_x) + img[y_rounded_down,x_rounded_up] *ratio_x 
    interpolate_x2 = img[y_rounded_up,x_rounded_down] * (1-ratio_x) + img[y_rounded_up,x_rounded_up]*ratio_x

    interpolate_value = interpolate_x1*(1-ratio_y)+ interpolate_x2*ratio_y

    return interpolate_value

def BilinearInterpolation(img, scale):
    new_img = np.zeros((int(img.shape[0]*scale), int(img.shape[1]*scale)),dtype=np.float32)
    for i in range(new_img.shape[0]):
        for j in range(new_img.shape[1]):
            x = i/scale
            y = j/scale
            if x > img.shape[0] - 1 :
                x = img.shape[0] - 1
            if y > img.shape[1] - 1 :
                y = img.shape[1] - 1   
            new_img[i,j] = bilinear_single_inter(y,x,img)

    new_img *= 255 
    new_img = new_img.astype(np.uint8)
    return new_img


img = plt.imread("./inputs/P2/wait_for_me_daddy_lr.png")
new_img2 = BilinearInterpolation(img,2)
plt.imsave("./outputs/P02/b-BI-Result.png",new_img2,cmap="gray")

In [ ]:
def NearestValueInterpolation(img, scale):
    new_img = np.zeros((int(img.shape[0]*scale), int(img.shape[1]*scale)),dtype=np.float32)
    h,w = new_img.shape
    for i in range(h):
        for j in range(w):
            x = i/scale
            y = j/scale
            
            if x > img.shape[0] - 1 :
                x = img.shape[0] - 1
            if y > img.shape[1] - 1 :
                y = img.shape[1] - 1   
                
            bilinear_value = bilinear_single_inter(y,x,img)
            start_i = int(i/scale)
            start_j = int(j/scale)
            
            target_dis = np.inf
            
            for row in range(start_i,min(img.shape[0],start_i+2)):
                for col in range(start_j,min(img.shape[1],start_j+2)):
                    if target_dis > abs(bilinear_value-img[row,col]):
                        target_dis = abs(bilinear_value-img[row,col])
                        new_img[i,j] = img[row,col]

            
    new_img *= 255 
    new_img = new_img.astype(np.uint8)
    return new_img

img = plt.imread("./inputs/P2/wait_for_me_daddy_lr.png")
new_img3 = NearestValueInterpolation(img,2)
plt.imsave("./outputs/P02/c-NVI-Result.png",new_img3,cmap="gray")


In [ ]:
print(cv2.PSNR(new_img1[:,:],GT[:,:]),
cv2.PSNR(new_img2[2:498,2:448],GT[2:498,2:448]),
cv2.PSNR(new_img3[2:498,2:448],GT[2:498,2:448]))

In [ ]:
img = plt.imread("./inputs/P2/wait_for_me_daddy_lr.png")
img = cv2.resize(img,(400,500),interpolation=cv2.INTER_LINEAR)
img *= 255
img = img.astype(np.uint8)
cv2.PSNR(img,GT)

In [ ]:
imgs = []
shifts = []
distances = []
img = plt.imread("./inputs/P2/wait_for_me_daddy_[+0.1,-0.5].png")
imgs.append(img)
shifts.append([0.5,0.1])
distances.append(1/sqrt(.5**2+.1**2))

img = plt.imread("./inputs/P2/wait_for_me_daddy_[+0.6,+0.2].png")
imgs.append(img)
shifts.append([-0.2,0.6])
distances.append(1/sqrt(.2**2+.6**2))


img = plt.imread("./inputs/P2/wait_for_me_daddy_[-0.2,-0.3].png")
imgs.append(img)
shifts.append([0.3,-0.2])
distances.append(1/sqrt(.3**2+.2**2))

img = plt.imread("./inputs/P2/wait_for_me_daddy_[-0.3,+0.4].png")
imgs.append(img)
shifts.append([-0.4,-0.3])
distances.append(1/sqrt(.4**2+.1**3))

def PreProcIndex(x,y,index,_sh):
    global shifts
    
    x += shifts[index][0]
    y += shifts[index][1]
    
    if x > _sh[0] - 1 :
        x = _sh[0] - 1
    if y > _sh[1] - 1 :
        y = _sh[1] - 1  
        
    if x < 0 :
        x = 0
    if y < 0 :
        y = 0
        
    return x,y


distances = [float(_)/sum(distances) for _ in distances]

def NonUniFormInterpolation(imgs, scale):
    new_img = np.zeros((int(imgs[0].shape[0]*scale), int(imgs[0].shape[1]*scale)),dtype=np.float32)
    global distances
    
    for i in range(new_img.shape[0]):
        for j in range(new_img.shape[1]):
                
            inters = []
            for index in range(len(imgs)):
                x = i/scale
                y = j/scale
                x,y = PreProcIndex(x,y,index,imgs[index].shape)
                value = bilinear_single_inter(y,x,imgs[index])
                value *= distances[index]
                inters.append(value)                
              
            new_img[i,j] = sum(inters)

    new_img *= 255 
    new_img = new_img.astype(np.uint8)
    return new_img


new_img4 = NonUniFormInterpolation(imgs,2)
plt.imsave("./outputs/P02/d-NUFI-Result.png",new_img4,cmap="gray")

In [ ]:
results = [new_img1,new_img2,new_img3,new_img4]
tags = ["Nearest-neighbor interpolation","Bilinear interpolation","Nearest neighbor value interpolation","Non-uniform interpolation"]

for index in range(4):
    plt.subplot(1,2,1)
    plt.imshow(GT,cmap="gray")
    plt.axis("off")
    plt.title("Ground Truth")
    
    plt.subplot(1,2,2)
    plt.imshow(results[index],cmap="gray")
    plt.axis("off")
    plt.title(tags[index]+"| PSNR: "+str(round(cv2.PSNR(GT,results[index]),3)))
    
    plt.tight_layout()
    plt.savefig("./outputs/P02/{0}-report.png".format(tags[index]))
    plt.show()

In [ ]:
norm = [float(i)/sum(distances) for i in distances]